# Preprocess 

### Set up binary file path

In [ ]:
prefetch="/mnt/data/kailu/Apps/sratoolkit/bin/prefetch"
fastdump="/mnt/data/kailu/Apps/sratoolkit/bin/fastq-dump"
trim="/mnt/data/kailu/Apps/Trimmomatic-0.39/trimmomatic-0.39.jar"
featurecount="/mnt/data/kailu/Apps/subread-2.0.3-source/bin/featureCounts"

#run the pipeline for every sample
# main_folder="/mnt/data/kailu/STAR_example/flash_seq_raw" #everything stored under main folder
input_sraList="$main_folder/SRR_Acc_List_PBMC_flash_seq_SE.txt"
output_mappingrate="$main_folder/mappingRateResults.txt"
#anno_gtf="/mnt/data/kailu/STAR_example/gencode_download/gencode.v41.primary_assembly.annotation.gtf"
anno_gtf="/mnt/data/kailu/STAR_example/ensembl_mod/Homo_sapiens.GRCh38.107.exon.gtf"


## Trim 

In [ ]:
## trim
!java -jar {trim} SE {main_folder}/{line}/{line}.fastq.gz {main_folder}/{line}/{line}.trim.fastq.gz ILLUMINACLIP:/mnt/data/kailu/Apps/Trimmomatic-0.39/adapters/TruSeq3-SE.fa:2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36 	
print("$line trimmed Success")

## Alignment

In [ ]:
### alignment
!STAR --runThreadN 4 \
        --genomeDir /mnt/data/kailu/STAR_example/ensembl_mod_indx/ \
        --readFilesIn {fastq_path}/{ID_SAMPLE}.trim.fastq.gz \
        --readFilesCommand gunzip -c \
        --outSAMtype BAM SortedByCoordinate \
        --outFileNamePrefix {main_folder}/{line}/{line}.

## Count

In [ ]:
#count the exon
!{featurecount} -t exon -f -O -J -M \
                -a {anno_gtf} \
                -o {main_folder}/{line}/{line}.count.txt \
                {main_folder}/{line}/${line}.Aligned.sortedByCoord.out.bam


#featurecounts - gene count
!{featureCounts} -t exon -O -J -M \
    -a /mnt/data/kailu/STAR_example/ensembl_mod/Homo_sapiens.GRCh38.107.exon.gtf \
    -o ./12_exon_gene_count/$ID_SAMPLE.gene.count.txt \
    ./11_single_exon_bam/filtered.TAG_CB:Z_$ID_SAMPLE-1.bam

#featurecounts - junction count
!{featureCounts} -t exon -f -O -J -M \
    -a /mnt/data/kailu/STAR_example/ensembl_mod/Homo_sapiens.GRCh38.107.exon.gtf \
    -o ./13_exon_junction_count/{ID_SAMPLE}.count.txt \
    ./11_single_exon_bam/filtered.TAG_CB:Z_{ID_SAMPLE}-1.bam